# Importando dependências

In [1]:
import json 
import pandas as pd 
from pandas.io.json import json_normalize 
import warnings
import numpy as np

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Acessando dados json

In [2]:
dados = pd.read_json('https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json')
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'Yes', 'Dependents': 'Yes', 'tenure': 9}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'No', 'TechSupport': 'Yes', 'StreamingTV': 'Yes', 'StreamingMovies': 'No'}","{'Contract': 'One year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}"
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 9}","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'No', 'TechSupport': 'No', 'StreamingTV': 'No', 'StreamingMovies': 'Yes'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'No', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 59.9, 'Total': '542.4'}}"
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 4}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'Yes', 'TechSupport': 'No', 'StreamingTV': 'No', 'StreamingMovies': 'No'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Electronic check', 'Charges': {'Monthly': 73.9, 'Total': '280.85'}}"
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partner': 'Yes', 'Dependents': 'No', 'tenure': 13}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'Yes', 'TechSupport': 'No', 'StreamingTV': 'Yes', 'StreamingMovies': 'Yes'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Electronic check', 'Charges': {'Monthly': 98.0, 'Total': '1237.85'}}"
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Partner': 'Yes', 'Dependents': 'No', 'tenure': 3}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'No', 'TechSupport': 'Yes', 'StreamingTV': 'Yes', 'StreamingMovies': 'No'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 83.9, 'Total': '267.4'}}"


## Acessando dados para análise posterior [concatenação]

In [3]:
# acessando dados dos dicionários com json_normalize
customer = json_normalize(dados['customer'])
phone = json_normalize(dados['phone'])
internet = json_normalize(dados['internet'])
account = json_normalize(dados['account'])
cust_churn = dados[['customerID', 'Churn']]

# Concatenando DataFrames anteriores em um único
dados = pd.concat([cust_churn, customer, phone, internet, account], axis=1)

# Primeira Vizualização dos dados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


Os dados apresentam `7257` observações, e `21` colunas.

In [7]:
# Verificando dados nulas
dados.isna().sum()

customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64

Aparentemente não há dados nulos.

In [8]:
# Dados duplicados
print("Quantidade de dados duplicados: " + str(dados.duplicated().sum()))

Quantidade de dados duplicados: 0


In [10]:
# verificando valores únicos constantes em cada coluna:
for column in dados:
    print('{} :\n{}'.format(column, dados[column].unique()))
    print('''####''')

customerID :
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
####
Churn :
['No' 'Yes' '']
####
gender :
['Female' 'Male']
####
SeniorCitizen :
[0 1]
####
Partner :
['Yes' 'No']
####
Dependents :
['Yes' 'No']
####
tenure :
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
####
PhoneService :
['Yes' 'No']
####
MultipleLines :
['No' 'Yes' 'No phone service']
####
InternetService :
['DSL' 'Fiber optic' 'No']
####
OnlineSecurity :
['No' 'Yes' 'No internet service']
####
OnlineBackup :
['Yes' 'No' 'No internet service']
####
DeviceProtection :
['No' 'Yes' 'No internet service']
####
TechSupport :
['Yes' 'No' 'No internet service']
####
StreamingTV :
['Yes' 'No' 'No internet service']
####
StreamingMovies :
['No' 'Yes' 'No internet service']
####
Contract :
['One year' 'Month-to-month'

In [12]:
# Com a análise das colunas acima é possível verificar que a coluna CHURN, apresenta valores de strings vazias:
dados.Churn.value_counts()

No     5174
Yes    1869
        224
Name: Churn, dtype: int64

In [14]:
print('O valor de dados vazios representa {:.2f}% em relação aos dados inteiros.'.format(dados.Churn.value_counts()[2]/dados.shape[0]))

O valor de dados vazios representa 0.03% em relação aos dados inteiros.


In [17]:
# Excluindo dados vazios:
dados = dados[~(dados['Churn'] == "")]
print('Observações resultantes: {}'.format(dados.shape[0]))

# Há modos mais simples de realizar a exclusão dos dados, mas por alguma razão eles não estavam funcionando.

Observações resultantes: 7043


#### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa 
* `gender`: gênero (masculino e feminino) 
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico 
* `MultipleLines`: assisnatura de mais de uma linha de telefone 
* `InternetService`: assinatura de um provedor internet 
* `OnlineSecurity`: assinatura adicional de segurança online 
* `OnlineBackup`: assinatura adicional de backup online 
* `DeviceProtection`: assinatura adicional de proteção no dispositivo 
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo 
* `StreamingMovies`: assinatura de streaming de filmes 
* `Contract`: tipo de contrato
* `PaperlessBilling`: se o cliente prefere receber online a fatura
* `PaymentMethod`: forma de pagamento
* `Charges.Monthly`: total de todos os serviços do cliente por mês
* `Charges.Total`: total gasto pelo cliente

# Transformando dados

In [22]:
# transformando dados em numéricos
dados['Charges.Total'] = pd.to_numeric(dados['Charges.Total'],downcast="integer", errors='coerce')

# após a transformação dos dados numéricos, apareceram dados incoerentes:
print("{:.3f}%".format(dados['Charges.Total'].isna().sum() / dados['Charges.Total'].shape[0]))

# e em razão da insignificância para o todo, resolvo excluí-los.

# Excluindo dados inconsistentes:
dados = dados[dados['Charges.Total'].isna()==False]

0.002%


## Tradução dos valores e colunas

In [26]:
# dicionário responsável pela tradução das colunas:
traduz_coluna = {'customerID': 'ID Cliente', 'gender': 'gênero', 'SeniorCitizen': 'idoso', 'Partner': 'Parceiro',
          'Dependents': 'Dependentes', 'tenure': 'Meses contrato', 'PhoneService': 'Serviço telefone', 'MultipleLines': 'Linhas multiplas',
          'InternetService': 'Serviço Internet', 'OnlineSecurity': 'Segurança Online', 'OnlineBackup': 'Backup Online', 'DeviceProtection': 'Dispositivo_Proteção',
          'TechSupport': 'Suporte Técnico', 'StreamingTV': 'Tv Streaming', 'StreamingMovies': 'Filmes Streaming', 'Contract':'Contrato', 'PaperlessBilling': 'Conta Papel', 
                 'PaymentMethod': 'Método pagamento', 'Charges.Monthly':'Cobrança Mensal','Charges.Total':'Cobrança Total'}

# Traduzindo colunas
dados.rename(columns=traduz_coluna, inplace=True)

In [27]:
# Categorizando dados binários:
dados['idoso'].replace({0: 'Não', 1: 'Sim'}, inplace=True)

In [28]:
# Traduzindo valores
tradução = {'No': 'Não', 'Yes': 'Sim', 'Male': 'Homem', 'Female': 'Mulher', 
              'Fiber optic': 'Fibra óptica', 'No internet service': 'Sem serviço', 
             'One year': 'Um ano', 'Month-to-month': 'Mês a mês', 'Two year': 'Dois anos',
             'Mailed check': 'Correspondência', 'Electronic check': 'Eletrônico', 
            'Credit card (automatic)': 'Cartão de Crédito', 'Bank transfer (automatic)': 'Transferência Bancária',
            'No phone service': 'Sem serviço'}


dados.replace(tradução, inplace=True)

In [30]:
# Criando coluna Cobrança diária
dados['Cobrança diária'] = dados['Cobrança Mensal']/30

In [31]:
# Exportando dados para arquivo CSV
dados.to_csv('Tratamento_alura_voz.csv', index=False)